In [1]:
!pip3 install pandas
!pip3 install seaborn
!pip3 install matplotlib
!pip3 install requests
!pip3 install xgboost
!pip3 install sklearn
!pip3 install numpy
!pip3 install mlflow
!pip3 install networkx
!pip3 install pyvis

  Using cached xgboost-1.7.3-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl (1.8 MB)
  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the enviro

In [2]:
# IMPORT LIBRARIES
import pandas as pd
import seaborn as sns 
import networkx
import matplotlib.pyplot as plt
from requests import get
from matplotlib import pyplot as plt
from datetime import datetime
import time
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import math
import mlflow
import pyvis
# from arango import ArangoClient

In [3]:
data = pd.read_json('/Users/marissaposner/Downloads/Telegram Desktop/briq-merged.json')
data['datetime']=pd.to_datetime(data['time'],unit='s')
#remove rows with null address because its just an artifact
data = data[data.recipient!='0x0000000000000000000000000000000000000000000000000000000000000000']
data.head()

,contract,name,time,block,hash,sender,recipient,value,datetime
0,0x01435498bf393da86b4733b9264a86b58a42b31f8d8b...,BRIQ,1669115906,10708,0x0595ba40ff386dee1514da6d64f348b0d950329b4188...,0x00000000000000000000000000000000000000000000...,0x0004b6767aea46adba81b673177c30dcad9cfc719aaf...,3.190302e+75,2022-11-22 11:18:26
1,0x01435498bf393da86b4733b9264a86b58a42b31f8d8b...,BRIQ,1669116041,10709,0x060400f1765d37e79d3a16ddaee7629cc10c30060e35...,0x00000000000000000000000000000000000000000000...,0x0449ff402a7496a773148c5ab0a087f36f60b1d85662...,2.176370e+75,2022-11-22 11:20:41
2,0x01435498bf393da86b4733b9264a86b58a42b31f8d8b...,BRIQ,1669116624,10711,0x077a30057d230b645a8a0a912b805d696cd97c14db7a...,0x00000000000000000000000000000000000000000000...,0x05a3143c9e3c16a317fc9e76b668ac5d573a51484a19...,1.971719e+75,2022-11-22 11:30:24
3,0x01435498bf393da86b4733b9264a86b58a42b31f8d8b...,BRIQ,1669116624,10711,0x07c04b21c1b8fdd6c2269c3c2e4d2df583f63890aa55...,0x00000000000000000000000000000000000000000000...,0x075f2b7695afc8c45cffae439201052511e52bd0b398...,5.472013e+74,2022-11-22 11:30:24
4,0x01435498bf393da86b4733b9264a86b58a42b31f8d8b...,BRIQ,1669116873,10712,0x0775c7a492f59c124c649754ea1e18b64ff03b997863...,0x00000000000000000000000000000000000000000000...,0x008a5cba00d6652011d809d9702aa44684187d72095e...,3.797527e+74,2022-11-22 11:34:33


In [ ]:
data.columns # columns
data.dtypes

In [ ]:
# Get shape of dataset
total_rows=len(data.index) # 0 for row
total_cols=len(data.columns) # 1 for column

print("Number of rows:", total_rows)
print("Number of columns:", total_cols)

In [ ]:
import networkx as nx
G = nx.from_pandas_edgelist(data.head(100), source = 'sender', target = 'recipient', edge_attr=True)
G.nodes

In [ ]:
data = pd.read_json('/Users/marissaposner/Downloads/Telegram Desktop/briq-merged.json')
G = nx.from_pandas_edgelist(data, source = 'sender', target = 'recipient', edge_attr=True)
G

In [ ]:
# Degree of Centrality
# Degree of Closeness
# Betweenness Centralit

from networkx import degree_centrality
degree_centrality(G)

In [4]:
from networkx import closeness_centrality

closeness_centrality(G)

NameError: name 'G' is not defined

In [26]:
from datetime import timedelta

unique_senders = data['sender'].unique()
time_diffs = {}
total_count_transactions = {}
number_of_unique_recipients = {}
all_data = {}
for i in unique_senders:
    sender_times=[]
    count_transactions = 0
    unique_recipients=[]
    for index, row in data.iterrows():
        if row['sender']==i:
            count_transactions+=1
            sender_times.append(row['datetime'])
            unique_recipients.append(row['recipient'])
    #dedupe and sort the times in ascending order
    sender_times = sorted(list(sender_times))
    num_unique_recipients = len(np.unique(unique_recipients))
    max_time = max(sender_times)
    min_time = min(sender_times)
    time_diff = (max_time - min_time).seconds
    all_data[i] = {'time_difference': time_diff,'count_transactions': count_transactions, 'num_unique_recipients': num_unique_recipients}
    time_diffs[i] = time_diff
    total_count_transactions[i] = count_transactions
    number_of_unique_recipients[i] = num_unique_recipients
# number_of_unique_recipients

In [27]:
import seaborn as sns
# s = pd.Series(time_diffs, name='time_difference')
# s.index.name = 'sender'
# s.reset_index()
# td=pd.DataFrame(s)
value_key_pairs = ((value, key) for (key,value) in time_diffs.items())
sorted_value_key_pairs = sorted(value_key_pairs, reverse=True)
sorted_value_key_pairs

td = pd.DataFrame(sorted_value_key_pairs, columns=['time_difference', 'sender'])
# cm = sns.light_palette((260, 75, 60),input="husl", as_cmap=True)
# s = td.style.background_gradient(cmap=cm,high=True,low=True)
# s

In [28]:
# total_count_transactions
value_key_pairs = ((value, key) for (key,value) in total_count_transactions.items())
sorted_value_key_pairs = sorted(value_key_pairs, reverse=True)
sorted_value_key_pairs
count_transactions = pd.DataFrame(sorted_value_key_pairs, columns=['total_count_transactions', 'sender'])

# td=pd.DataFrame.from_dict(time_diffs, orient='index',columns=['sender','time_difference'])
# cm = sns.light_palette((260, 75, 60),input="husl", as_cmap=True)
# s = td.style.background_gradient(cmap=cm,high=True,low=True)
# s
count_transactions

,total_count_transactions,sender
0,798,0x009153c3344ecf28213b730c016751e838374da62a75...
1,265,0x00000000000000000000000000000000000000000000...
2,69,0x0411f475ea06286ce6cb6d11787ccae6d2b0748a18b2...
3,62,0x01f56b8b404909641d4a7845d7e3ae47e7291ce8e524...
4,35,0x074fcbb1006e5a0fb043d6ed72759983dd050d914f5c...
...,...,...
143,1,0x0073bcb0f57f340972bdd8bb2488aca80eca8197d9ba...
144,1,0x0067f1324966df0de4bcf65ffe8e08a9a0877af77e32...
145,1,0x006541c67be85b13b43bddb90ee66b24690e4eca7702...
146,1,0x00579ed214135f8ed807110d417e47171bdbf9f2c828...


In [18]:
# number_of_unique_recipients
value_key_pairs = ((value, key) for (key,value) in number_of_unique_recipients.items())
sorted_value_key_pairs = sorted(value_key_pairs, reverse=True)
sorted_value_key_pairs

num_unique_recipients = pd.DataFrame(sorted_value_key_pairs, columns=['num_unique_recipients', 'sender'])
# td=pd.DataFrame.from_dict(time_diffs, orient='index',columns=['sender','time_difference'])
# cm = sns.light_palette((260, 75, 60),input="husl", as_cmap=True)
# s = td.style.background_gradient(cmap=cm,high=True,low=True)
# s
num_unique_recipients

,num_unique_recipients,sender
0,89,0x00000000000000000000000000000000000000000000...
1,26,0x074fcbb1006e5a0fb043d6ed72759983dd050d914f5c...
2,23,0x0411f475ea06286ce6cb6d11787ccae6d2b0748a18b2...
3,14,0x01cea635ebd5fcf9e6f95fe8ff43dd54856d30a45926...
4,12,0x051102de9c077ba26886ea9533c9d6bdbc55550b1cd8...
...,...,...
143,1,0x0067f1324966df0de4bcf65ffe8e08a9a0877af77e32...
144,1,0x006541c67be85b13b43bddb90ee66b24690e4eca7702...
145,1,0x00579ed214135f8ed807110d417e47171bdbf9f2c828...
146,1,0x001146a4d56cffa6a97a62fc221af2eb00b1e1d90278...


In [16]:
# td = td.reset_index()

std_1 = round(td['time_difference'].mean() + 1 * td['time_difference'].std(),0)
std_2 = round(td['time_difference'].mean() + 2 * td['time_difference'].std(),0)
std_3 = round(td['time_difference'].mean() + 3 * td['time_difference'].std(),0)
std_4 = round(td['time_difference'].mean() + 4 * td['time_difference'].std(),0)

time_diff_scores = {}
for index, row in td.iterrows():
    if row['time_difference']<=std_1:
        time_diff_scores[row['sender']]=0
    if row['time_difference']>=std_1 and row['time_difference']<std_2:
        time_diff_scores[row['sender']]=0.5
    if row['time_difference']>=std_2:
        time_diff_scores[row['sender']]=1
    
# time_diff_scores

In [20]:
std_1 = round(num_unique_recipients['num_unique_recipients'].mean() + 1 * num_unique_recipients['num_unique_recipients'].std(),0)
std_2 = round(num_unique_recipients['num_unique_recipients'].mean() + 2 * num_unique_recipients['num_unique_recipients'].std(),0)
std_3 = round(num_unique_recipients['num_unique_recipients'].mean() + 3 * num_unique_recipients['num_unique_recipients'].std(),0)
std_4 = round(num_unique_recipients['num_unique_recipients'].mean() + 4 * num_unique_recipients['num_unique_recipients'].std(),0)

num_unique_recipients_scores = {}
for index, row in num_unique_recipients.iterrows():
    if row['num_unique_recipients']<=std_1:
        num_unique_recipients_scores[row['sender']]=0
    if row['num_unique_recipients']>=std_1 and row['num_unique_recipients']<std_2:
        num_unique_recipients_scores[row['sender']]=0.5
    if row['num_unique_recipients']>=std_2:
        num_unique_recipients_scores[row['sender']]=1

# num_unique_recipients_scores

In [31]:
std_1 = round(count_transactions['total_count_transactions'].mean() + 1 * count_transactions['total_count_transactions'].std(),0)
std_2 = round(count_transactions['total_count_transactions'].mean() + 2 * count_transactions['total_count_transactions'].std(),0)
std_3 = round(count_transactions['total_count_transactions'].mean() + 3 * count_transactions['total_count_transactions'].std(),0)
std_4 = round(count_transactions['total_count_transactions'].mean() + 4 * count_transactions['total_count_transactions'].std(),0)

count_transation_scores = {}
for index, row in count_transactions.iterrows():
    if row['total_count_transactions']<=std_1:
        count_transation_scores[row['sender']]=0
    if row['total_count_transactions']>=std_1 and row['total_count_transactions']<std_2:
        count_transation_scores[row['sender']]=0.5
    if row['total_count_transactions']>=std_2:
        count_transation_scores[row['sender']]=1
# count_transation_scores

{'0x009153c3344ecf28213b730c016751e838374da62a7572899f0e638094cd446f': 1,
 '0x0000000000000000000000000000000000000000000000000000000000000000': 1,
 '0x0411f475ea06286ce6cb6d11787ccae6d2b0748a18b20b9e15172cbe52aabdef': 0,
 '0x01f56b8b404909641d4a7845d7e3ae47e7291ce8e52427cbdf174cc2287416a0': 0,
 '0x074fcbb1006e5a0fb043d6ed72759983dd050d914f5c4ce86c4794a132ad75cc': 0,
 '0x01cea635ebd5fcf9e6f95fe8ff43dd54856d30a45926586753b47899ed36d6cf': 0,
 '0x06661c6a44a0ad85eeb455179c792ca665eba4dc9c21b093a78a535b1bc243ff': 0,
 '0x051102de9c077ba26886ea9533c9d6bdbc55550b1cd88969b747df22a9c595ac': 0,
 '0x04cda5884a3a4c89261dff18b14dd8e4a4a7038d9f8bffe14b744e6e3ecd57b7': 0,
 '0x0042a1e72aa8d66179b7134017f0801e8c9462ae1ecbf00ee9b16672e035a1b2': 0,
 '0x0365316b44bc410b1b8678665731e5f5d5ba95ad5f91a6f487995ec5b05769cf': 0,
 '0x06581b3fb9af12cf7d11af40c7ed029a8b38961bd699efb3f4b5d1b4827fc6d9': 0,
 '0x0724102a5654d669ba7ce25e420e733ddb409b753598ab0883ad476cf9faed7b': 0,
 '0x0028331f5c89594131ec67457b58c73616

In [38]:


for k, v in num_unique_recipients_scores.items():
    print(k, v)

0x0000000000000000000000000000000000000000000000000000000000000000 1
0x074fcbb1006e5a0fb043d6ed72759983dd050d914f5c4ce86c4794a132ad75cc 1
0x0411f475ea06286ce6cb6d11787ccae6d2b0748a18b20b9e15172cbe52aabdef 1
0x01cea635ebd5fcf9e6f95fe8ff43dd54856d30a45926586753b47899ed36d6cf 0.5
0x051102de9c077ba26886ea9533c9d6bdbc55550b1cd88969b747df22a9c595ac 0.5
0x0439e93a21ed825e6bc05ba501ff61cd749650600eff86c14e3cbe0cce2843af 0.5
0x01f56b8b404909641d4a7845d7e3ae47e7291ce8e52427cbdf174cc2287416a0 0
0x07b393627bd514d2aa4c83e9f0c468939df15ea3c29980cd8e7be3ec847795f0 0
0x06581b3fb9af12cf7d11af40c7ed029a8b38961bd699efb3f4b5d1b4827fc6d9 0
0x075f2b7695afc8c45cffae439201052511e52bd0b39852f10fe8b957e82e408e 0
0x06d51deeb8acdad615a68e2eb39e3807eff04ddf4c83e8b94ba22af26ac9202d 0
0x06661c6a44a0ad85eeb455179c792ca665eba4dc9c21b093a78a535b1bc243ff 0
0x040d88f70eee82d065c43892fa12e1c562faabd4fc0db9b7a7161a3f1ab6109f 0
0x04cda5884a3a4c89261dff18b14dd8e4a4a7038d9f8bffe14b744e6e3ecd57b7 0
0x010884171baf1914edc28d7afb

In [45]:
# lists = [count_transation_scores, num_unique_recipients_scores, time_diff_scores]
unique_senders = data['sender'].unique()

sender_scores = {}
for sender in unique_senders:
    temp_sender_score = 0
    temp_sender_count = 0 
    for k, v in count_transation_scores.items():
        if k==sender:
            temp_sender_count+=1
            temp_sender_score +=v
    for k, v in num_unique_recipients_scores.items():
        if k==sender:
            temp_sender_count+=1
            temp_sender_score +=v
    for k, v in time_diff_scores.items():
        if k==sender:
            temp_sender_count+=1
            temp_sender_score +=v
    total_score = temp_sender_score / temp_sender_count
    sender_scores[sender]=total_score
# sender_scores


In [47]:
import networkx as nx
G = nx.from_pandas_edgelist(data, source = 'sender', target = 'recipient', edge_attr=True)
G.nodes

NodeView(('0x0000000000000000000000000000000000000000000000000000000000000000', '0x0004b6767aea46adba81b673177c30dcad9cfc719aaf6487c03f8319389a1548', '0x0449ff402a7496a773148c5ab0a087f36f60b1d856621eec4e789041cfbc1ccb', '0x05a3143c9e3c16a317fc9e76b668ac5d573a51484a19cbc0d97aadc9bf84d89a', '0x075f2b7695afc8c45cffae439201052511e52bd0b39852f10fe8b957e82e408e', '0x008a5cba00d6652011d809d9702aa44684187d72095ea2017b74a652b3751037', '0x02e1b826ddb432c0021484b5a6935492de4fc06f4caa712a96f75fb650b4695e', '0x05778cafcec4e2e10e659aa0a8b272278243e41abf050b7f4dc7e48a9b342f0b', '0x0516fc05cc24654411b301d9226e7cc0b28c25ade4824c0ec7ce8af1ed82118b', '0x006541c67be85b13b43bddb90ee66b24690e4eca7702507d8337b0bb4fbbc95a', '0x006cb9685ee69092d0a502663b46b2da2b3634602ddbfedccd55532364b52c68', '0x005c8c4a46de99124a7f7629977fcb1d42e3087babc9d28239117ed43e61e1e9', '0x0035dd8e05a39fb93ae3508049d0d6b171be67a9d6f633c1b67acbb291032a78', '0x013a086a56d89f688eb9dda3a7c29bd4075a7af02a06fa3cf20545ababbfad79', '0x0245e8b